In [1]:
import string
import joblib
from os import path

import numpy as np
import pandas as pd
from scipy import sparse
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm

from utils import ls, cat

tqdm.pandas()

In [2]:
DATA_DIR = '../../data'
OUTPUT_DIR = '../../data'

ls(DATA_DIR)

DIR      ../../data/stopwords-ru
  2.1 GB ../../data/lenta-ru-news.csv
 20.0  B ../../data/roles.csv
 22.2 MB ../../data/train_lemmas.csv
 22.7 MB ../../data/train_tokens.csv


In [3]:
TRAIN_PATH = f'{DATA_DIR}/lenta-ru-news.csv'
NUM_ROWS = 10000

df_train = pd.read_csv(TRAIN_PATH, nrows=NUM_ROWS)
display(df_train.head(3))
display(df_train.shape)

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17


(10000, 6)

## Features

In [4]:
text_cols = ['title', 'text']

df_train[text_cols].head()

,title,text
0,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...
1,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про..."
2,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в..."
3,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...
4,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю..."


## Tokenized Text

In [5]:
# !pip install spacy
# python -m spacy download ru_core_news_md

In [6]:
import spacy

nlp = spacy.load('ru_core_news_md', disable=['tagger', 'attribute_ruler', 'senter', 'parser', 'ner'])

In [7]:
train_tokens = df_train[text_cols].fillna('') \
    .progress_applymap(nlp)
    
train_tokens

100%|█████████████████████████████████████| 20000/20000 [05:57<00:00, 55.97it/s]


,title,text
0,"(1914, ., Русские, войска, вступили, в, , пре...","(Бои, у, Сопоцкина, и, Друскеник, закончились,..."
1,"(1914, ., Празднование, столетия, М.Ю., Лермон...","(Министерство, народного, просвещения, ,, в, в..."
2,"(1914, ., Das, ist, Nesteroff, !)","(Штабс, -, капитан, П., Н., Нестеров, на, днях..."
3,"(1914, ., Бульдог, -, гонец, под, Льежем)","(Фотограф, -, корреспондент, Daily, Mirror, ра..."
4,"(1914, ., Под, Люблином, пойман, швабский, зверь)","(Лица, ,, приехавшие, в, Варшаву, из, Люблина,..."
...,...,...
9995,"(На, границе, Дагестана, и, , Чечни, будут, с...","(Глава, МВД, Дагестана, приказал, нарядам, мил..."
9996,"(Путин, пересадит, губернаторов, в, , Госсовет)","(Представители, Кремля, ведут, активный, диало..."
9997,"(В, Москву, съедутся, клоуны, со, , всего, мира)","(В, Москве, 20, июля, открывается, фестиваль, ..."
9998,"(Старшая, Уильямс, выигрывает, чемпионский, ти...","( , Чисто, американским, получился, финал, Уим..."


In [9]:
TRAIN_LEMMAS_PATH = f'{OUTPUT_DIR}/train_lemmas.csv'


def is_alphanum(token):
    return not token.is_punct \
        and not token.is_currency \
        and not token.is_digit \
        and not token.is_punct \
        and not token.is_oov \
        and not token.is_space \
        and not token.is_stop \
        and not token.like_num \
        and not token.pos_ == "PROPN"

train_tokens \
    .progress_applymap(lambda x: ' '.join(t.lemma_ for t in x if is_alphanum(t))) \
    .to_csv(TRAIN_LEMMAS_PATH)

ls(OUTPUT_DIR, path.basename(TRAIN_LEMMAS_PATH))

100%|██████████████████████████████████| 20000/20000 [00:01<00:00, 12207.19it/s]

 13.9 MB ../../data/train_lemmas.csv


In [10]:
pd.read_csv(TRAIN_LEMMAS_PATH, index_col=[0])

,title,text
0,войско вступить предел,закончиться отступление германец приблизиться ...
1,столетие отложить,народный просвещение вид происходить чрезвычай...
2,ist,капитан день увидеть район лететь расположение...
3,гонец,корреспондент рассказывать случай порадует дру...
4,поймать швабский зверь,приехать передавать доставить швабский зверь в...
...,...,...
9995,граница стрелять водитель нарушитель,приказать наряд милиция приграничный блокпост ...
9996,губернатор,вести активный диалог губернатор лидер думский...
9997,съедутся клоун мир,июль открываться фестиваль клоунада смотр колл...
9998,выигрывать чемпионский титул,американский получиться финал теннисный турнир...
